## **Assignment one to six**

In [0]:
!wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz

--2019-12-17 16:15:30--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2144903 (2.0M) [application/x-gzip]
Saving to: ‘kddcup.data_10_percent.gz’

kddcup.data_10_perc 100%[===================>]   2.04M  1.35MB/s    in 1.5s    

2019-12-17 16:15:32 (1.35 MB/s) - ‘kddcup.data_10_percent.gz’ saved [2144903/2144903]



In [0]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,745

In [0]:
!wget https://datahack.analyticsvidhya.com/contest/black-friday/download/train-file

--2019-12-18 05:07:56--  https://datahack.analyticsvidhya.com/contest/black-friday/download/train-file
Resolving datahack.analyticsvidhya.com (datahack.analyticsvidhya.com)... 35.154.159.229
Connecting to datahack.analyticsvidhya.com (datahack.analyticsvidhya.com)|35.154.159.229|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /oidc/authenticate/?next=/contest/black-friday/download/train-file [following]
--2019-12-18 05:07:57--  https://datahack.analyticsvidhya.com/oidc/authenticate/?next=/contest/black-friday/download/train-file
Reusing existing connection to datahack.analyticsvidhya.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://id.analyticsvidhya.com/openid/authorize?client_id=161754&redirect_uri=http%3A%2F%2Fdatahack.analyticsvidhya.com%2Foidc%2Fcallback%2F&nonce=qm3w8HSdCkzSS1erUmGmoPu1bfbG5ql5&response_type=code&scope=openid+email+profile&state=7nmxqjbAU9SJr9ZLoKNuOljZPM12lxlp [following]
--2019-12-18 05:07:57--  https:/

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>


Creating a RDD from a file

In [0]:
data_file = "./train-file"
raw_data = sc.textFile(data_file)

In [0]:
raw_data.count()

147

In [0]:
raw_data.take(5)

['<!DOCTYPE html>',
 '',
 '<html lang="en-us">',
 '<head>',
 '  <meta charset="utf-8">']

Creating and RDD using parallelize

In [0]:
a = range(100)

data = sc.parallelize(a)

In [0]:
data.count()

100

In [0]:
data.take(5)

[0, 1, 2, 3, 4]

The filter transformation

In [0]:
normal_raw_data = raw_data.filter(lambda x: 'normal.' in x)

In [0]:
from time import time
t0 = time()
normal_count = normal_raw_data.count()
tt = time() - t0
print("There are {} 'normal' interactions".format(normal_count))
print("Count completed in {} seconds".format(round(tt,3)))

There are 0 'normal' interactions
Count completed in 0.137 seconds


The map transformation

In [0]:
from pprint import pprint
csv_data = raw_data.map(lambda x: x.split(","))
t0 = time()
head_rows = csv_data.take(5)
tt = time() - t0
print("Parse completed in {} seconds".format(round(tt,3)))
pprint(head_rows[0])

Parse completed in 0.095 seconds
['<!DOCTYPE html>']


In [0]:
t0 = time()
head_rows = csv_data.take(100000)
tt = time() - t0
print("Parse completed in {} seconds".format(round(tt,3)))

Parse completed in 0.196 seconds


Using map and predefined functions

In [0]:
def parse_interaction(line):
    elems = line.split(",")
    tag = elems[41]
    return (tag, elems)

key_csv_data = raw_data.map(parse_interaction)
head_rows = key_csv_data.take(5)
pprint(head_rows[0])

The collect action

In [0]:
t0 = time()
all_raw_data = raw_data.collect()
tt = time() - t0
print("Data collected in {} seconds".format(round(tt,3)))

Data collected in 0.065 seconds


In [0]:
# get data from file
data_file = "./train-file"
raw_data = sc.textFile(data_file)

# parse into key-value pairs
key_csv_data = raw_data.map(parse_interaction)

# filter normal key interactions
normal_key_interactions = key_csv_data.filter(lambda x: x[0] == "normal.")

# collect all
t0 = time()
all_normal = normal_key_interactions.collect()
tt = time() - t0
normal_count = len(all_normal)
print("Data collected in {} seconds".format(round(tt,3)))
print("There are {} 'normal' interactions".format(normal_count))


The sample transformation

In [0]:
raw_data_sample = raw_data.sample(False, 0.1, 1234)
sample_size = raw_data_sample.count()
total_size = raw_data.count()
print("Sample size is {} of {}".format(sample_size, total_size))

Sample size is 18 of 147


In [0]:
from time import time

# transformations to be applied
raw_data_sample_items = raw_data_sample.map(lambda x: x.split(","))
sample_normal_tags = raw_data_sample_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
sample_normal_tags_count = sample_normal_tags.count()
tt = time() - t0

sample_normal_ratio = sample_normal_tags_count / float(sample_size)
print("The ratio of 'normal' interactions is {}".format(round(sample_normal_ratio,3)) )
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.0
Count done in 0.118 seconds


In [0]:
# transformations to be applied
raw_data_items = raw_data.map(lambda x: x.split(","))
normal_tags = raw_data_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
normal_tags_count = normal_tags.count()
tt = time() - t0

normal_ratio = normal_tags_count / float(total_size)
print("The ratio of 'normal' interactions without sampling is {}".format(round(normal_ratio,3)))
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions without sampling is 0.0
Count done in 0.108 seconds



The takeSample action

In [0]:
t0 = time()
raw_data_sample = raw_data.takeSample(False, 400000, 1234)
normal_data_sample = [x.split(",") for x in raw_data_sample if "normal." in x]
tt = time() - t0

normal_sample_size = len(normal_data_sample)

normal_ratio = normal_sample_size / 400000.0
print("The ratio of 'normal' interactions is {}".format(normal_ratio))
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.0
Count done in 0.15 seconds


Getting attack interactions using subtract

In [0]:
normal_raw_data = raw_data.filter(lambda x: "normal." in x)

In [0]:
attack_raw_data = raw_data.subtract(normal_raw_data)

In [0]:
from time import time

# count all
t0 = time()
raw_data_count = raw_data.count()
tt = time() - t0
print("All count in {} secs".format(round(tt,3)))

All count in 0.131 secs


In [0]:
# count normal
t0 = time()
normal_raw_data_count = normal_raw_data.count()
tt = time() - t0
print("Normal count in {} secs".format(round(tt,3)))

Normal count in 0.11 secs


In [0]:
# count attacks
t0 = time()
attack_raw_data_count = attack_raw_data.count()
tt = time() - t0
print("Attack count in {} secs".format(round(tt,3)))

Attack count in 0.857 secs


In [0]:
print("There are {} normal interactions and {} attacks, \
from a total of {} interactions".format(normal_raw_data_count,attack_raw_data_count,raw_data_count))

There are 0 normal interactions and 147 attacks, from a total of 147 interactions



Protocol and service combinations using cartesian

In [0]:
csv_data = raw_data.map(lambda x: x.split(","))
protocols = csv_data.map(lambda x: x[1]).distinct()
protocols.collect()

In [0]:
services = csv_data.map(lambda x: x[2]).distinct()
services.collect()

In [0]:
product = protocols.cartesian(services).collect()
print "There are {} combinations of protocol X service".format(len(product))

Inspecting interaction duration by tag

In [0]:
# parse data
csv_data = raw_data.map(lambda x: x.split(","))

# separate into different RDDs
normal_csv_data = csv_data.filter(lambda x: x[41]=="normal.")
attack_csv_data = csv_data.filter(lambda x: x[41]!="normal.")

In [0]:
normal_duration_data = normal_csv_data.map(lambda x: int(x[0]))
attack_duration_data = attack_csv_data.map(lambda x: int(x[0]))

In [0]:
total_normal_duration = normal_duration_data.reduce(lambda x, y: x + y)
total_attack_duration = attack_duration_data.reduce(lambda x, y: x + y)

print "Total duration for 'normal' interactions is {}".\
    format(total_normal_duration)
print "Total duration for 'attack' interactions is {}".\
    format(total_attack_duration)

In [0]:
normal_count = normal_duration_data.count()
attack_count = attack_duration_data.count()

print "Mean duration for 'normal' interactions is {}".\
    format(round(total_normal_duration/float(normal_count),3))
print "Mean duration for 'attack' interactions is {}".\
    format(round(total_attack_duration/float(attack_count),3))

Using aggregate

In [0]:
normal_sum_count = normal_duration_data.aggregate(
    (0,0), # the initial value
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # combine value with acc
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])) # combine accumulators
)

print "Mean duration for 'normal' interactions is {}".\
    format(round(normal_sum_count[0]/float(normal_sum_count[1]),3))

In [0]:
attack_sum_count = attack_duration_data.aggregate(
    (0,0), # the initial value
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # combine value with acc
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])) # combine accumulators
)

print "Mean duration for 'attack' interactions is {}".\
    format(round(attack_sum_count[0]/float(attack_sum_count[1]),3))


Creating a pair RDD for interaction types

In [0]:
csv_data = raw_data.map(lambda x: x.split(","))
key_value_data = csv_data.map(lambda x: (x[41], x)) # x[41] contains the network interaction tag

In [0]:
key_value_data.take(1)


Data aggregations with key/value pair RDDs

In [0]:
key_value_duration = csv_data.map(lambda x: (x[41], float(x[0]))) 
durations_by_key = key_value_duration.reduceByKey(lambda x, y: x + y)

durations_by_key.collect()

In [0]:
counts_by_key = key_value_data.countByKey()
counts_by_key


Using combineByKey

In [0]:
sum_counts = key_value_duration.combineByKey(
    (lambda x: (x, 1)), # the initial value, with value x and count 1
    (lambda acc, value: (acc[0]+value, acc[1]+1)), # how to combine a pair value with the accumulator: sum value, and increment count
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # combine accumulators
)

sum_counts.collectAsMap()

In [0]:
duration_means_by_type = sum_counts.map(lambda (key,value): (key, round(value[0]/value[1],3))).collectAsMap()

# Print them sorted
for tag in sorted(duration_means_by_type, key=duration_means_by_type.get, reverse=True):
    print tag, duration_means_by_type[tag]

# Black Friday Assignment

In [0]:
from pyspark.sql import SQLContext
sqlContext=SQLContext(sc)

In [0]:
!wget https://datahack.analyticsvidhya.com/contest/black-friday/download/train-file

--2019-12-18 09:50:00--  https://datahack.analyticsvidhya.com/contest/black-friday/download/train-file
Resolving datahack.analyticsvidhya.com (datahack.analyticsvidhya.com)... 35.154.159.229
Connecting to datahack.analyticsvidhya.com (datahack.analyticsvidhya.com)|35.154.159.229|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /oidc/authenticate/?next=/contest/black-friday/download/train-file [following]
--2019-12-18 09:50:06--  https://datahack.analyticsvidhya.com/oidc/authenticate/?next=/contest/black-friday/download/train-file
Reusing existing connection to datahack.analyticsvidhya.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://id.analyticsvidhya.com/openid/authorize?state=wmXgROamokkk752CnFy8izeDn14yO3m6&scope=openid+email+profile&nonce=FI5PtFBJbkp7cEIfiXsSeLduZkFV9aXf&redirect_uri=http%3A%2F%2Fdatahack.analyticsvidhya.com%2Foidc%2Fcallback%2F&response_type=code&client_id=161754 [following]
--2019-12-18 09:50:06--  https:/

In [0]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 54kB/s 
     |████████████████████████████████| 204kB 45.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=20520d49976078f41cd951a616f008a17cb107753ceb729847c73f0d5ef0f4d4
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
train = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("./train.csv")

How to see datatype of columns?

In [0]:
train.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



First n obs

In [0]:
train.head(5)

[Row(User_ID=1000001, Product_ID='P00069042', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=3, Product_Category_2=None, Product_Category_3=None, Purchase=8370),
 Row(User_ID=1000001, Product_ID='P00248942', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=1, Product_Category_2=6, Product_Category_3=14, Purchase=15200),
 Row(User_ID=1000001, Product_ID='P00087842', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=12, Product_Category_2=None, Product_Category_3=None, Purchase=1422),
 Row(User_ID=1000001, Product_ID='P00085442', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=12, Product_Category_2=14, Product_Category_3=None, Purchase=1057),
 Row(User_ID=1000002, Product_ID='P0

In [0]:
#counting rows
train.count()

550068

In [0]:
#No of Columns
len(train.columns)


12

In [0]:
#Name of cols
train.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase']

In [0]:
#Summary Stats
train.describe().show()

+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|   Age|       Occupation|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|            550068|    550068|550068|550068|           550068|       550068|                    550068|             550068|            550068|            376430|            166821|           550068|
|   mean|1003028.8424013031|      null|  null|  null|8.076706879876669|         null|         1.468494139793958|0.40965298835780306| 5.404270017525106| 9.84232925112238

In [0]:
#Selecting COls
train.select('User_ID','Age').show(5)
train.select('Occupation','Age').show(5)


+-------+----+
|User_ID| Age|
+-------+----+
|1000001|0-17|
|1000001|0-17|
|1000001|0-17|
|1000001|0-17|
|1000002| 55+|
+-------+----+
only showing top 5 rows

+----------+----+
|Occupation| Age|
+----------+----+
|        10|0-17|
|        10|0-17|
|        10|0-17|
|        10|0-17|
|        16| 55+|
+----------+----+
only showing top 5 rows



In [0]:
#No of distinct users
train.select('User_ID').distinct().count()

5891

In [0]:
#Crosstabs
train.crosstab('Age', 'Occupation').show(5)


+--------------+-----+----+-----+----+----+----+----+----+----+-----+----+----+----+----+----+-----+----+----+-----+---+----+
|Age_Occupation|    0|   1|   10|  11|  12|  13|  14|  15|  16|   17|  18|  19|   2|  20|   3|    4|   5|   6|    7|  8|   9|
+--------------+-----+----+-----+----+----+----+----+----+----+-----+----+----+----+----+----+-----+----+----+-----+---+----+
|          0-17| 2134| 387|10951|  18| 237|  15|  93|   0|   0|   35|   0| 807| 144|   0|   0|  113|   0|   0|  139| 29|   0|
|         46-50| 4488|7089|    0|1584|2491| 631|1445| 854|3032| 4662|1124| 261|2124|2699|1599|  129|1187|2561| 6664|549| 528|
|         18-25| 9095|3820| 1649| 717|4585|   0|4388| 906|1816| 3944|1085|2500|4364|5445|1860|48241|1450|1144| 2078| 14| 559|
|         36-45|13393|9501|  170|2732|6848| 427|5590|2585|7572|10252|1527|1008|5183|7508|4126| 1747|3066|4822|18762| 98|3096|
|           55+| 1722|3139|  134| 143| 322|4870|1335| 432|1963| 1558| 112| 217| 812| 801| 812|    0|  15| 660| 2075|16

In [0]:
#No duplicates
train.select('Age','Gender').dropDuplicates().show()

+-----+------+
|  Age|Gender|
+-----+------+
|51-55|     F|
|18-25|     M|
| 0-17|     F|
|46-50|     M|
|18-25|     F|
|  55+|     M|
|  55+|     F|
|36-45|     M|
|26-35|     F|
| 0-17|     M|
|36-45|     F|
|51-55|     M|
|26-35|     M|
|46-50|     F|
+-----+------+



In [0]:
#Drop rows with null values
train.dropna().count()

166821

In [0]:
train.fillna(-1).show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|                -1|                -1|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
only

In [0]:
#filter
train.filter(train.Purchase > 15000).count()

110523

In [0]:
#finding mean
train.groupby('Age').agg({'Purchase': 'mean'}).show()

+-----+-----------------+
|  Age|    avg(Purchase)|
+-----+-----------------+
|18-25|9169.663606261289|
|26-35|9252.690632869888|
| 0-17|8933.464640444974|
|46-50|9208.625697468327|
|51-55|9534.808030960236|
|36-45|9331.350694917874|
|  55+|9336.280459449405|
+-----+-----------------+



In [0]:
#Usig group by
train.groupby('Age').count().show()

+-----+------+
|  Age| count|
+-----+------+
|18-25| 99660|
|26-35|219587|
| 0-17| 15102|
|46-50| 45701|
|51-55| 38501|
|36-45|110013|
|  55+| 21504|
+-----+------+



In [0]:
#SOrt dataframe 
train.orderBy(train.Purchase.desc()).show(5)

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1003160| P00052842|     M|26-35|        17|            C|                         3|             0|                10|                15|              null|   23961|
|1002272| P00052842|     M|26-35|         0|            C|                         1|             0|                10|                15|              null|   23961|
|1001474| P00052842|     M|26-35|         4|            A|                         2|             1|                10|                15|              null|   23961

In [0]:
#Adding cols in DataFrame
train.withColumn('Purchase_new', train.Purchase /2.0).select('Purchase','Purchase_new').show(5)

+--------+------------+
|Purchase|Purchase_new|
+--------+------------+
|    8370|      4185.0|
|   15200|      7600.0|
|    1422|       711.0|
|    1057|       528.5|
|    7969|      3984.5|
+--------+------------+
only showing top 5 rows



In [0]:
#Dropping cols
train.drop('Comb').columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase']

In [0]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
F1 = udf(lambda x: '-1' if x in not_found_cat else x, StringType())

In [0]:
#Writing SQL

train.registerTempTable('train_table')
sqlContext.sql('select Product_ID from train_table').show(5)

+----------+
|Product_ID|
+----------+
| P00069042|
| P00248942|
| P00087842|
| P00085442|
| P00285442|
+----------+
only showing top 5 rows

